# **ECON871: RICARDIAN MODEL**

<blockquote>This is a learning practice both for python and international economics. We build a simple 3-country Ricardian Model from Eaton & Kortum (2002).</blockquote>

Parameters of interest are:
<ul>
   <li> N is number of countries</li>
   <li> K is number of commodities</li>
   <li> &theta; is the dispersion of Frechet distribution</li>
   <li> &sigma; is elasticity</li>
   <li> &tau; is the trade cost matrix</li>
   <li> T is the technology array</li>
   <li> L is the labor enodowment matrix</li>
   <li> wage is wage matrix</li>
   <li> welfare is calculated by wage/L</li>
   <li> P is price index array</li>
   <li> Trade_Volume is the trade value matrix</li>
   <li> Share is the trade value share matrix</li>
</ul>

__Steps to Implement Ricardian Model__

<ol>
    <li>Generate an instance</li>
    <li>Change the Parameter as you want</li>
    <li>Generally, use sovle() function to solve for a wage vector</li>
    <li>Run report() to get all results</li>
<ol>

In [1]:
import numpy as np
from scipy.optimize import fsolve 
import math
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

class Ricardian:
    
     
    def __init__(self, N=3,                   # CNTY number
                       K=200000,              # Commodity number
                       theta = 4,             # Frechet Parameter2
                       sig = 2,               # Elasticity
                       tau = np.ones((3, 3)), # Trade cost
                       set_seed = 4):         # seed for productivity

        self.N, self.K, self.theta, self.sig, self.seed = N, K, theta, sig, set_seed
        self.L, self.T, self.tau = np.ones(N), 1.5*np.ones(N), np.ones((N, N))
        self.wage, self.welfare= np.ones((N-1, 1)), np.ones(N)
        self.P, self.Trade_Volume, self.Share = np.ones((N, 1)), np.ones((N, N)), np.ones((N, N))
    
    def set_Labor(self,L):
        self.L = L
    def set_T(self, T):
        self.T = T
    def set_theta(self, theta):
        self.theta = theta
    def set_tau(self, tau):
        self.tau = tau
    def set_wage(self, wage):
        self.wage = wage
        
    #Generate Frechet random number   
    def cdf_inv(self, u,T):
        return (-1/T * np.log(u))**(-1/self.theta)
    
    def productivity_gen(self):
        #Return matrix z
        z = np.ones((self.K, self.N))
        #Draw uniform random numbers.
        np.random.seed(seed = self.seed)
        uni = np.random.rand(self.K, self.N)
        #Convert to Frechet randoms, make it simpler?
        for j in range(0,self.N):
            for i in range(0,self.K):
                z[i,j] = self.cdf_inv(uni[i,j], self.T[j])
        return z
    
     # Define a function to compute pni and pn return 
     # a vector of prices and suppliers
    def trade_decision(self):
        z = self.productivity_gen()
        #Add normalized wage1
        wage = np.insert(self.wage, 0, 1)
    
        #Price Index and Spending Share
        P = np.zeros((self.N, 1))
        Trade_Volume = np.zeros((self.N, self.N))
        Share = np.zeros((self.N,self. N))
    
        #For each country, get its cheapest supplier and price
        for n in range(0, self.N): 
            index = np.argmin(wage * self.tau[n, :] / z, axis = 1)
            # price index for country n
            p = (wage * self.tau[n, :] / z)[range(self.K), index]
            P[n, 0] = (np.sum( 1 / self.K * p ** (1 - self.sig))) ** (1/(1 - self.sig))
        
            #spending = wage[n] * self.L[n] * (p / P[n, 0]) ** (1 - self.sig)
            spending = wage[n] * self.L[n] * (p / P[n, 0]) ** (1 - self.sig)
        
            for j in range(0, self.N):
                Trade_Volume[n, j] = np.sum(spending [index == j])
            
        row_sum = [sum(Trade_Volume[n,:]) for n in range(self.N)]
        
        for i in range(0, self.N):
            Share[i,:] = Trade_Volume[i,:] / row_sum[i]
            
        self.P, self.Trade_Volume, self.Share = P, Trade_Volume, Share
     
    ##N-1 D equations
    def f(self, w):
        self.set_wage(w) 
        self.trade_decision()
        y = np.empty(self.N-1)
        for n in range(self.N-1):
            y[n] = sum(self.Trade_Volume[n,:]) - sum(self.Trade_Volume[:,n])
        return y
     
    def solve(self):
        self.wage = fsolve(self.f, self.wage)
    
    ##Welfare calculation
    def welfare_get(self):
        self.trade_decision()
        wage = np.insert(self.wage, 0, 1)
        for n in range(0, self.N):
            self.welfare[n] = wage[n] / self.P[n]
            
    ##Report all results
    def report(self):
        np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
        self.trade_decision()
        self.welfare_get()
        wage = np.insert(self.wage, 0, 1)
        print('The equilibrium wage vector is:', wage)
        print('\nThe trade share matrix is:\n', self.Share)
        print('\nWelfare (w/P) is:', self.welfare)


<b>a. Simple and symmetric.</b>Let there be no trade costs, i.e., $\tau_{ni} = 0$, and let $T_i = 1.5$ for all $i$. Report the equilibrium bilateral trade share matrix. (An element of the matrix is $\pi_{ ni}$, the share of total spending in $n$ on goods from $i$.) The solution to this model is trivial, so this is a good place to first check that our programs are working.

In [2]:
r1 = Ricardian()

r1.trade_decision()

print('The expenditure matrix is:')
print(r1.Share)

The expenditure matrix is:
[[0.335 0.332 0.333]
 [0.335 0.332 0.333]
 [0.335 0.332 0.333]]


<b>b. Symmetric geography. </b>Now introduce iceberg trade costs. Let $\tau_{ ni} = 0.1$ for each $n \neq i$, and keep the remaining parameters as in part (a). Report the bilateral trade share matrix.

In [3]:
# Create the trade cost matrix
tau = np.ones((3, 3))*1.1
for n in range(0,3):
    tau[n, n] = 1.0
    
r2 = Ricardian()
r2.set_tau(tau)
r2.trade_decision()

print('\nThe expenditure matrix is:')
print(r2.Share)


The expenditure matrix is:
[[0.424 0.288 0.289]
 [0.290 0.422 0.289]
 [0.291 0.288 0.422]]


<b>c. Asymmetric geography. </b>Countries have identical technologies, $T_i = 1.5$ for all $i$, and $\theta = 4$. Country 3, however, is 'far away' from countries 1 and 2: $\tau_{12} = \tau_{ 21} = 1.05$ and $\tau_{ 13} = \tau_{ 31} = \tau_{ 32} = \tau_{ 23} = 1.3$. Report the equilibrium bilateral trade share matrix. (An element of the matrix is $\pi_{ni}$, the share of total spending in $n$ on goods from $i$.) Report an index of welfare in each country, $w_i /P_i$ , where $P_i$ is the CES aggregate price index.

In [4]:
tau[0,:] = [1.0, 1.05, 1.3]
tau[1,:] = [1.05, 1.0, 1.3]
tau[2,:] = [1.3, 1.3, 1.0]

r3 = Ricardian()
r3.set_tau(tau)
r3.solve()
r3.report()

The equilibrium wage vector is: [1.000 0.998 0.963]

The trade share matrix is:
 [[0.449 0.369 0.182]
 [0.369 0.448 0.182]
 [0.190 0.188 0.622]]

Welfare (w/P) is: [1.660 1.657 1.527]


<b>d. Technological progress. </b>Let $T_2 = 3$, and keep the remaining parameters as in part (c). This is a technological improvement in country 2. Report the bilateral trade share matrix.

Compare welfare in this economy with welfare in the economy in part (c). Discuss your findings in the context of how an increase in technology in one country benefits other countries.

In [5]:
r4 = Ricardian()
r4.T[1] = 3

tau[0,:] = [1.0, 1.05, 1.3]
tau[1,:] = [1.05, 1.0, 1.3]
tau[2,:] = [1.3, 1.3, 1.0]
r4.set_tau(tau)

r4.solve()
r4.report()

The equilibrium wage vector is: [1.000 1.149 0.960]

The trade share matrix is:
 [[0.426 0.398 0.176]
 [0.347 0.480 0.174]
 [0.183 0.208 0.609]]

Welfare (w/P) is: [1.682 1.938 1.535]


<b>e. Frechet dispersion. </b>Now change $\theta = 8$, and keep the remaining parameters as in part (c) (i.e., change $T_2 = 1.5$).Report the bilateral trade share matrix. Compare welfare in this economy with welfare in the economy in part (c). What is the intuition for this result? How does the change in $\theta$ affect countries 1 and 2 compared to country 3? Why? 

In [6]:
r3.set_theta(8)
r3.solve()
r3.report()

The equilibrium wage vector is: [1.000 0.999 0.975]

The trade share matrix is:
 [[0.547 0.371 0.082]
 [0.371 0.548 0.082]
 [0.085 0.083 0.832]]

Welfare (w/P) is: [1.237 1.236 1.173]


It is worth noting that higher &theta; results more welfare loss for CNTY1 and 2. This is because higher &theta; makes the productivity distribution more concentrated. Intuitively, the productivity difference between countries is smaller, thus more loss are incurred when one of the countries raises its trade cost.

<h2>More on OOP:

one of the advantage of OOP is that it facilitates counterfactual analysis. In the following example, we compare the welfare loss when countries turn from frictionless trade to autarky.

In [7]:
r_fl = Ricardian()
r_a = Ricardian()
tau = math.inf * np.ones((3, 3))
for n in range(3):
    tau[n, n] = 1
r_a.set_tau(tau)
r_a.solve()
r_a.welfare_get()
r_fl.welfare_get()
print("\nWelfare Loss between a frictionless and autarky world:\n ", r_a.welfare / r_fl.welfare)


Welfare Loss between a frictionless and autarky world:
  [0.760 0.759 0.759]


Therefore, under our default setting, welfare of autarky is approximately 75% that of frictionless trade.